In [2]:
# Turn on autoload for debugging
%load_ext autoreload
%autoreload 2

In [62]:
from dateutil.relativedelta import relativedelta
import datetime
from owslib.wfs import WebFeatureService
import logging
import geopandas as gpd
import pandas as pd
from dotenv import load_dotenv
import os
import glob
import zipfile, io
import fiona
import json
import requests

from src.utils import check_countries
import src.geo_utils as geo
import src.data_import as imp

In [87]:
load_dotenv('/home/jovyan/work/.env')
os.environ['PROJ_LIB'] = '/opt/conda/share/proj'
PATH_RAW = '../../data/temp/'
log = logging.getLogger(__name__)


In [84]:
with open('../../config/sources.json') as f:
    sources = json.load(f)

In [17]:
zipfiles = [file for file in sources if 'zip' in file.get('filetype')]

In [44]:
def create_nl_zipfiles(zipfiles):
    
    today = datetime.date.today() - relativedelta(months=1) #(uncomment if current month hasn't been uploaded yet
    year = today.year
    month_name = today.strftime('%b').lower()
    month_number = today.strftime('%m')
    
    nl_files = [file.get('url') for file in zipfiles if 'nl' in file.get('countries')]

    nl_dict = dict()
    # This is a bit of a hack. Would prefer to use f-strings
    for name, file in zip(nl_names, nl_files):
        file = file.replace('{year}', str(year)).replace('{month_number}', str(month_number)).replace('{month_name}', month_name)
        update = {name:file}
        nl_dict.update(update)
    
    return dict(nl_dict)

In [94]:
def download_zipfiles(countries, zipfiles):
    '''Downloads zipfiles
    Parameters:
    -----------
    countries : list
        List of countries'''
    
    for country in check_countries(countries):
        log.info(f'working on {country}')
        if country not in ['nl', 'uk', 'no']: # change this if zipfiles for other countries are available
            raise ValueError(f'No zipfiles available for {country}, please use function download_wfs')
        
        elif country in ['uk', 'no']:
            urls = [file.get('url') for file in zipfiles if country in file.get('countries')]

        elif country in ['nl']:
            urls = [value for value in create_nl_zipfiles(zipfiles).values()]
            
        path = f'{PATH_RAW}{country}/'

        if not os.path.exists(path):
            os.makedirs(path)

        for url in urls:
            r = requests.get(url)
            z = zipfile.ZipFile(io.BytesIO(r.content))
            z.extractall(path)
        
        # Write files to geopackage, this is not necessary per se, but makes life a little bit easier later on
        if country in ['nl', 'uk']:
            for file in glob.glob(f'{path}/*.shp'):
                if country == 'uk':
                    name = os.path.basename(file)[5:-9].lower()
                elif country == 'nl':
                    name = os.path.basename(file)[14:-8].lower()
                
                gdf = gpd.read_file(file)
                gdf = gdf[~gdf.geometry.isna()]
                
                geo.gdf_to_geopackage(gdf, country, name, path)
                log.info(f'wrote {country} to geopackage')
        

In [95]:
download_zipfiles(['nl', 'no', 'uk'], zipfiles)

CPLE_AppDefinedError: b'failed to prepare SQL: SELECT "fid", ST_MinX("geom"), ST_MaxX("geom"), ST_MinY("geom"), ST_MaxY("geom") FROM "offshore_fielddets_" WHERE "geom" NOT NULL AND NOT ST_IsEmpty("geom"): no such table: offshore_fielddets_'

Exception ignored in: 'fiona.ogrext.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 198, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'failed to prepare SQL: SELECT "fid", ST_MinX("geom"), ST_MaxX("geom"), ST_MinY("geom"), ST_MaxY("geom") FROM "offshore_fielddets_" WHERE "geom" NOT NULL AND NOT ST_IsEmpty("geom"): no such table: offshore_fielddets_'


RuntimeError: GDAL Error: sqlite3_exec(CREATE TABLE "offshore_fielddets_" ( "fid" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, "geom" GEOMETRY, "OBJECTID" INTEGER, "FIELDNAME" TEXT, "Shape_STAr" REAL, "Shape_STLe" REAL, "ID" REAL, "CLUSTERNAM" REAL, "CLUSTERNO" REAL, "STATUS" REAL, "WEBLINK" REAL, "CREATED_US" TEXT, "CREATED_DA" TEXT, "LAST_EDITE" TEXT, "LAST_EDI00" TEXT, "FIELDTYPE" REAL, "NAME_SHORT" REAL, "DISC_DATE" REAL, "DISC_WELL" REAL, "STAT_CODE" REAL, "PROD_DATE" REAL, "DEPTH_M" REAL, "DET_STATUS" REAL, "ORIG_OP" REAL, "ORIG_LIC" REAL, "CURR_OPER" REAL, "START_DATE" REAL, "END_DATE" REAL, "FIELDDATA" REAL, "ORIG_LIC_2" REAL, "ORIG_LIC_3" REAL, "ORIG_LIC_4" REAL, "ORIG_LIC_5" REAL, "OGA_COP" REAL, "CURR_LIC" REAL, "EQUITY" REAL, "CURR_LIC_2" REAL, "EQUITY2" REAL, "CURR_LIC_3" REAL, "EQUITY3" REAL, "CURR_LIC_4" REAL, "EQUITY4" REAL, "CURR_LIC_5" REAL, "EQUITY5" REAL, "CURR_LIC_6" REAL, "EQUITY6" REAL, "CURR_LIC_7" REAL, "EQUITY7" REAL, "CURR_LIC_8" REAL, "EQUITY8" REAL, "CURR_LI